## Prática Guiada Requests e Beautiful Soup

# Exemplo IMDB

**Raspando dados para mais de 2000 filmes**


Queremos analisar as distribuições das classificações de filmes do IMDB e do Metacritic para ver se encontramos algo interessante. Para fazer isso, primeiro vamos coletar dados para mais de 2000 filmes.

É essencial identificar o objetivo da nossa raspagem desde o início. Escrever um script de raspagem pode levar muito tempo, especialmente se quisermos raspar mais de uma página da web. Queremos evitar passar horas escrevendo um script que raspe dados que realmente não precisaremos.

**Trabalhando em quais páginas raspar**


Uma vez que estabelecemos nosso objetivo, precisamos identificar um conjunto eficiente de páginas para raspar.

Queremos encontrar uma combinação de páginas que requer um número relativamente pequeno de solicitações. Uma solicitação é o que acontece sempre que acessamos uma página da web. Nós "solicitamos" o conteúdo de uma página do servidor. Quanto mais pedidos fizermos, mais tempo o script precisará executar e maior será a sobrecarga do servidor.

Uma forma de obter todos os dados de que precisamos é compilar uma lista de nomes de filmes e usá-los para acessar a página da web de cada filme nos sites do IMDB e do Metacritic.
<br>
<img src='https://www.dataquest.io/blog/content/images/2017/12/option1.gif'>
<br>
Como queremos obter mais de 2.000 classificações do IMDB e do Metacritic, teremos que fazer pelo menos 4.000 solicitações. Se fizermos uma solicitação por segundo, nosso script precisará de um pouco mais de uma hora para fazer 4000 solicitações. Por isso, vale a pena tentar identificar maneiras mais eficientes de obter nossos dados.

Se explorarmos o site do IMDB, podemos descobrir uma maneira de reduzir o número de solicitações pela metade. As pontuações do Metacritic são mostradas na página do filme do IMDB, por isso podemos recapitular ambas as classificações com um único pedido:


<br>
<img src='https://www.dataquest.io/blog/content/images/option2.jpg'>
<br>
<br>
Se investigarmos o site do IMDB, poderemos descobrir a página mostrada abaixo. Ele contém todos os dados que precisamos para 50 filmes. Dado o nosso objetivo, isso significa que só teremos que fazer 40 pedidos, o que é 100 vezes menor que a nossa primeira opção. Vamos explorar mais esta última opção.

<img src='https://www.dataquest.io/blog/content/images/option3.jpg'>
<br>
<br>
**Identificando a estrutura do URL**
<br>
<br>
Nosso desafio agora é ter certeza de que entendemos a lógica do URL como as páginas que queremos para alterar a mudança. Se não conseguirmos entender essa lógica o suficiente para podermos implementá-la em código, chegaremos a um beco sem saída.
<br>
Se você for na página de pesquisa avançada do IMDB , você pode procurar filmes por ano:
<br>

<img src='https://www.dataquest.io/blog/content/images/advanced_search.png'>

Vamos navegar pelo ano de 2017, ordenar os filmes na primeira página por número de votos, depois mudar para a próxima página. Chegaremos a esta página da web , que tem este URL:


<br>

<img src='https://www.dataquest.io/blog/content/images/url.png'>
<br>
<br>

Na imagem acima, você pode ver que o URL tem vários parâmetros após o ponto de interrogação:

* `release_date` - Mostra apenas os filmes lançados em um ano específico.
* `sort` - Classifica os filmes na página. sort=num_votes,desctraduz para classificar por número de votos em ordem decrescente .
* `page` - Especifica o número da página.
* `ref_` - Leva-nos para a próxima página ou a anterior. A referência é a página em que estamos atualmente. adv_nxte adv_prvsão dois valores possíveis. Eles traduzem para avançar para a próxima página e avançar para a página anterior , respectivamente.

<br>
<br>
<br>

Se você navegar por essas páginas e observar o URL, perceberá que apenas os valores dos parâmetros mudam. Isso significa que podemos escrever um script para corresponder à lógica das alterações e fazer muito menos solicitações para coletar nossos dados.
<br>
<br>

Vamos começar a escrever o script, solicitando o conteúdo desta página web único: http://www.imdb.com/search/title?release_date=2017&sort=num_votes,desc&page=1. Na seguinte célula de código nós iremos:

Importe a get()função do requestsmódulo.
Atribua o endereço da página da web a uma variável nomeada url.
Solicite ao servidor o conteúdo da página da Web usando get()e armazene a resposta do servidor na variável response.
Imprima uma pequena parte do responseconteúdo acessando seu .textatributo ( responseagora é um Responseobjeto).

In [1]:
from requests import get

url = 'http://www.imdb.com/search/title?release_date=2017&sort=num_votes,desc&page=1'

response = get(url)
print(response.text[:500])




<!DOCTYPE html>
<html
    xmlns:og="http://ogp.me/ns#"
    xmlns:fb="http://www.facebook.com/2008/fbml">
    <head>
         
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">

    <meta name="apple-itunes-app" content="app-id=342792525, app-argument=imdb:///?src=mdot">



        <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>

<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle"


# Entendendo a estrutura HTML de uma única página

<br>
<img src='https://cdn-images-1.medium.com/max/1000/0*ETFzXPCNHkPpqNv_.png'>
<br>
Como você pode ver na primeira linha response.text, o servidor nos enviou um documento HTML. Este documento descreve a estrutura geral dessa página da Web, juntamente com seu conteúdo específico (que é o que torna essa página específica única).

Todas as páginas que queremos copiar têm a mesma estrutura geral. Isso implica que eles também têm a mesma estrutura HTML geral. Então, para escrever nosso script, será suficiente entender a estrutura HTML de apenas uma página. Para fazer isso, usaremos as ferramentas de desenvolvedor do navegador .

Se você usa o Chrome , clique com o botão direito do mouse em um elemento da página da web que lhe interessa e clique em Inspecionar . Isso levará você diretamente para a linha HTML que corresponde a esse elemento:

<br>
<img src='https://www.dataquest.io/blog/content/images/inspecthtml.png'>
<br>

Clique com o botão direito do mouse no nome do filme e, em seguida, clique com o botão esquerdo do mouse em Inspecionar . A linha HTML destacada em cinza corresponde ao que o usuário vê na página da Web como o nome do filme.

Você também pode fazer isso usando o Firefox e o Safari DevTools.

Observe que todas as informações de cada filme, incluindo o pôster, estão contidas em uma divtag.

<br>
<img src='https://www.dataquest.io/blog/content/images/one_container.jpg'>
<br>

Existem muitas linhas HTML aninhadas em cada divtag. Você pode explorá-las clicando nas pequenas setas cinzas à esquerda das linhas HTML correspondentes a cada uma delas div. Dentro dessas tags aninhadas, encontraremos as informações de que precisamos, como a classificação de um filme.


<br>
<img src='https://www.dataquest.io/blog/content/images/movie_rating.jpg'>
<br>

Há 50 filmes exibidos por página, portanto, deve haver um divcontêiner para cada um. Vamos extrair todos esses 50 contêineres analisando o documento HTML de nossa solicitação anterior.

# Usando BeautifulSoup para analisar o conteúdo HTML
Para analisar nosso documento HTML e extrair os 50 divcontêineres, usaremos um módulo Python chamado BeautifulSoup , o módulo de raspagem da Web mais comum para Python.

Na seguinte célula de código nós iremos:

- Importe o `BeautifulSoupcriador` da classe do pacote bs4.
- Analise `response.text` criando um `BeautifulSoup` objeto e atribua esse objeto a `html_soup`. O `html.parser` argumento indica que queremos fazer a análise usando o [analisador HTML do Python](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#specifying-the-parser-to-use).



In [ ]:
response

In [7]:
from bs4 import BeautifulSoup

html_soup = BeautifulSoup(response.text, 'html.parser')
html_soup


<!DOCTYPE html>

<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="app-id=342792525, app-argument=imdb:///?src=mdot" name="apple-itunes-app"/>
<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
<title>IMDb: Most Voted Titles Released 2017-01-01 to 2017-12-31 - IMDb</title>
<script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>
<script>
    if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
</script>
<lin

Antes de extrair os 50 contêineres, precisamos descobrir o que os diferencia de outros divelementos nessa página. Muitas vezes, a marca distintiva reside no class atributo . Se você inspecionar as linhas HTML dos contêineres de interesse, notará que o classatributo tem dois valores: lister-iteme mode-advanced. Essa combinação é exclusiva desses divcontêineres. Podemos ver que isso é verdade fazendo uma pesquisa rápida ( Ctrl + F). Temos 50 desses contêineres, então esperamos ver apenas 50 correspondências:


<br>
<img src='https://www.dataquest.io/blog/content/images/search.jpg'>
<br>

Agora vamos usar o find_all() método para extrair todos os divcontainers que possuem um classatributo lister-item mode-advanced:



In [8]:
movie_containers = html_soup.find_all('div', class_ = 'lister-item mode-advanced')
print(type(movie_containers))
print(len(movie_containers))

<class 'bs4.element.ResultSet'>
50


In [10]:
# for movie in movie_containers:
#     print(movie)
#     break

O `find_all()` retornou um `ResultSet` objeto que é uma lista contendo todos os 50 divs que nos interessam.

Agora vamos selecionar apenas o primeiro container e extrair, por sua vez, cada item de interesse:

- O nome do filme
- O ano do lançamento.
- A classificação do IMDB.
- O Metascore.
- O número de votos.

<br>
<img src='https://www.dataquest.io/blog/content/images/datapoints.jpg'>
<br>


# Extraindo os dados para um único filme
Podemos acessar o primeiro contêiner, que contém informações sobre um único filme, usando notação de lista em movie_containers.



In [12]:
first_movie = movie_containers[0]
first_movie

<div class="lister-item mode-advanced">
<div class="lister-top-right">
<div class="ribbonize" data-caller="filmosearch" data-tconst="tt3315342"></div>
</div>
<div class="lister-item-image float-left">
<a href="/title/tt3315342/?ref_=adv_li_i"> <img alt="Logan" class="loadlate" data-tconst="tt3315342" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BYzc5MTU4N2EtYTkyMi00NjdhLTg3NWEtMTY4OTEyMzJhZTAzXkEyXkFqcGdeQXVyNjc1NTYyMjg@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/G/01/imdb/images/nopicture/large/film-184890147._CB470041630_.png" width="67"/>
</a> </div>
<div class="lister-item-content">
<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<a href="/title/tt3315342/?ref_=adv_li_tt">Logan</a>
<span class="lister-item-year text-muted unbold">(2017)</span>
</h3>
<p class="text-muted ">
<span class="certificate">16</span>
<span class="ghost">|</span>
<span class="runtime">137 min</span>
<span class="ghost">|<

Como você pode ver, o conteúdo HTML de um contêiner é muito longo. Para descobrir a linha HTML específica para cada ponto de dados, usaremos o DevTools novamente.


# O nome do filme

Começamos com o nome do filme e localizamos sua linha HTML correspondente usando o DevTools. Você pode ver que o nome está contido em uma tag de âncora ( a). Esta tag é aninhada dentro de uma tag de cabeçalho (h3). A h3 tag está aninhada em uma div tag. Este div é o terceiro do divsaninhado no container do primeiro filme. Nós armazenamos o conteúdo desse contêiner na first_movie variável.

<br>
<img src='https://www.dataquest.io/blog/content/images/movie_name.jpg'>
<br>

first_movieé um Tag objeto , e as várias tags HTML dentro dele são armazenadas como seus atributos. Podemos acessá-los como se tivéssemos acesso a qualquer atributo de um objeto Python. No entanto, usar um nome de tag como um atributo selecionará apenas a primeira tag com esse nome. Se corrermos first_movie.div, só obtemos o conteúdo da primeira divtag:


In [13]:
first_movie.div

<div class="lister-top-right">
<div class="ribbonize" data-caller="filmosearch" data-tconst="tt3315342"></div>
</div>

Acessar a primeira tag âncora (**a**) não nos leva ao nome do filme. O primeiro **a** está em algum lugar dentro do segundo div:

In [16]:
first_movie.a

<a href="/title/tt3315342/?ref_=adv_li_i"> <img alt="Logan" class="loadlate" data-tconst="tt3315342" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BYzc5MTU4N2EtYTkyMi00NjdhLTg3NWEtMTY4OTEyMzJhZTAzXkEyXkFqcGdeQXVyNjc1NTYyMjg@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/G/01/imdb/images/nopicture/large/film-184890147._CB470041630_.png" width="67"/>
</a>

No entanto, acessar a primeira **h3** tag nos aproxima muito:

In [17]:
first_movie.h3

<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<a href="/title/tt3315342/?ref_=adv_li_tt">Logan</a>
<span class="lister-item-year text-muted unbold">(2017)</span>
</h3>

A partir daqui, podemos usar a notação de atributo para acessar o primeiro **a** dentro da **h3** tag:

In [15]:
first_movie.h3.a

<a href="/title/tt3315342/?ref_=adv_li_tt">Logan</a>

Agora tudo é apenas uma questão de acessar o texto de dentro dessa **a** :

In [16]:
first_name = first_movie.h3.a.text
first_name

'Logan'

# O ano do lançamento do filme
Continuamos com a extração do ano. Esses dados são armazenados na **span** abaixo do **a** que contém o nome.

<br>
<img src='https://www.dataquest.io/blog/content/images/year_name.png'>
<br>

A notação de pontos só acessará o primeiro spanelemento. Vamos procurar pela marca distintiva do segundo **span**. Nós vamos usar o `find()` método que é quase o mesmo que `find_all()`, exceto que só retorna a primeira partida. Na verdade, `find()` é equivalente a `find_all(limit = 1)`. O limit argumento limita a saída para a primeira correspondência.

A marca distintiva consiste nos valores lister-item-year text-muted unboldatribuídos ao classatributo. Então procuramos o primeiro **span** com esses valores dentro da **h3**:

In [18]:
first_year = first_movie.h3.find('span', class_ = 'lister-item-year text-muted unbold')
first_year

<span class="lister-item-year text-muted unbold">(2017)</span>

A partir daqui, basta acessar o texto usando a notação de atributo:

In [19]:
first_year = first_year.text
first_year

'(2017)'

Poderíamos facilmente limpar essa saída e convertê-la em um inteiro. Mas, se você explorar mais páginas, perceberá que, para alguns filmes, o ano aceita valores imprevisíveis como (2017) (I) ou (2015) (V). É mais eficiente fazer a limpeza depois da raspagem, quando saberemos todos os valores do ano.

# A classificação do IMDB
Agora nos concentramos em extrair a classificação do IMDB do primeiro filme.

Existem algumas maneiras de fazer isso, mas primeiro tentaremos a mais fácil. Se você inspecionar a classificação do IMDB usando DevTools, perceberá que a classificação está contida em uma **strong**.

<br>
<img src='https://www.dataquest.io/blog/content/images/imdb_rating.png'>
<br>


Vamos usar a notação de atributo e esperar que o primeiro **strong** também seja o que contenha a classificação.



In [19]:
first_movie.strong

<strong>8.1</strong>

Ótimo! Vamos acessar o texto, convertê-lo para o `float`tipo e atribuí-lo à variável `first_imdb`:

In [20]:
first_imdb = float(first_movie.strong.text)
first_imdb

8.1

# O Metascore
Se nós inspecionarmos o Metascore usando DevTools, notamos que podemos encontrá-lo dentro de uma **span**.

<br>
<img src='https://www.dataquest.io/blog/content/images/metascore.png'>
<br>

A notação de atributos claramente não é uma solução. Existem muitas **span** antes disso. Você pode ver um logo acima da **strong**. É melhor usarmos os valores distintivos do classatributo ( metascore favorable).

Observe que, se você copiar e colar esses valores da guia DevTools, haverá dois caracteres de espaço em branco entre metascoree favorable. Certifique-se de que haverá apenas um caractere de espaço em branco quando você passar os valores como argumentos. Caso contrário, `find()` não encontrará nada.



In [22]:
first_mscore = first_movie.find('span', class_ = 'metascore favorable')

first_mscore = int(first_mscore.text)
print(first_mscore)

77


O favorablevalor indica um alto Metascore e define a cor de fundo da classificação como verde. Os outros dois valores possíveis são unfavorablee mixed. O que é específico para todas as classificações do Metascore é apenas o metascorevalor. Este é o que vamos usar quando vamos escrever o script para a página inteira.

# O número de votos
O número de votos está contido em uma <span>tag. Sua marca distintiva é um nameatributo com o valor nv.

<br>
<img src='https://www.dataquest.io/blog/content/images/nr_votes.png'>
<br>


O name atributo é diferente do class atributo. Usando BeautifulSoup podemos acessar elementos por qualquer atributo. As funções `find()` e `find_all()` têm um parâmetro chamado `attrs`. Para isso, podemos passar os atributos e valores que procuramos como dicionário:

In [23]:
first_votes = first_movie.find('span', attrs = {'name':'nv'})
first_votes.text

'511,985'

Poderíamos usar a .text notação para acessar o <span>conteúdo da tag. Seria melhor se nós acessássemos o valor do data-valueatributo. Desta forma, podemos converter o ponto de dados extraído para um intsem ter que retirar uma vírgula.

Você pode tratar um Tagobjeto como um dicionário. Os atributos HTML são as chaves do dicionário. Os valores dos atributos HTML são os valores das chaves do dicionário. É assim que podemos acessar o valor do data-valueatributo:

In [49]:
first_votes['data-value']

'510160'

Vamos converter esse valor para um inteiro e atribuí-lo a first_votes:

In [50]:
first_votes = int(first_votes['data-value'])

É isso aí! Estamos agora em posição de escrever facilmente um script para criar uma única página.

O script para uma única página
Antes de juntar o que fizemos até agora, temos que ter certeza de que extrairemos os dados apenas dos contêineres que possuem um Metascore.

<br>
<img src='https://www.dataquest.io/blog/content/images/no_mscores.jpg'>
<br>

Precisamos adicionar uma condição para ignorar filmes sem um Metascore.

Usando DevTools novamente, vemos que a seção Metascore está contida em uma **div**. O class atributo tem dois valores: inline-blocke ratings-metascore. O distintivo é claro ratings-metascore.

<br>
<img src='https://www.dataquest.io/blog/content/images/metascore_yes.png'>
<br>


Podemos usar find() para pesquisar cada contêiner de filme por divter essa marca distinta. Quando find() não encontra nada, retorna um Noneobjeto. Podemos usar esse resultado em uma ifinstrução para controlar se um filme é copiado.

Vamos procurar na [página da web](https://www.imdb.com/search/title?release_date=2017&sort=num_votes,desc&page=1) para procurar por um contêiner de filme que não tenha um Metascore e ver o que find() retorna.

Importante: quando eu corri o seguinte código, o oitavo contêiner não tinha um Metascore. No entanto, este é um alvo em movimento, porque o número de votos muda constantemente para cada filme. Para obter as mesmas saídas que na próxima célula de código demonstrativo, você deve pesquisar um contêiner que não tenha um Metascore no momento em que estiver executando o código.

In [24]:
movie22_mscore = movie_containers[22].find('div', class_ = 'ratings-metascore')
type(movie22_mscore)

NoneType

In [ ]:
movie22_mscore

Agora vamos montar o código acima e comprimir o máximo possível, mas apenas na medida em que ele ainda é facilmente legível. No próximo bloco de código nós:

- Declare algumas listas de variáveis para ter algo para armazenar os dados extraídos.
- Faça um loop por cada contêiner movie_containers(a variável que contém todos os 50 contêineres de filme).
- Extraia os pontos de dados de interesse somente se o contêiner tiver um Metascore.

In [25]:
# Lists to store the scraped data in
names = []
years = []
imdb_ratings = []
metascores = []
votes = []

# Extract data from individual movie container
for container in movie_containers:

    # If the movie has Metascore, then extract:
    if container.find('div', class_ = 'ratings-metascore') is not None:

        # The name
        name = container.h3.a.text
        names.append(name)

        # The year
        year = container.h3.find('span', class_ = 'lister-item-year').text
        years.append(year)

        # The IMDB rating
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb)

        # The Metascore
        m_score = container.find('span', class_ = 'metascore').text
        metascores.append(int(m_score))

        # The number of votes
        vote = container.find('span', attrs = {'name':'nv'})['data-value']
        votes.append(int(vote))

In [26]:
len(votes)

47

Vamos verificar os dados coletados até o momento. Os pandas facilitam para nós ver se coletamos nossos dados com sucesso.

In [27]:
import pandas as pd

test_df = pd.DataFrame({'movie': names,
                       'year': years,
                       'imdb': imdb_ratings,
                       'metascore': metascores,
                       'votes': votes})
test_df

,movie,year,imdb,metascore,votes
0,Logan,(2017),8.1,77,511985
1,Mulher-Maravilha,(2017),7.5,76,439271
2,Dunkirk,(2017),8.0,94,421617
3,Star Wars: Os Últimos Jedi,(2017),7.2,85,418527
4,Guardiões da Galáxia Vol. 2,(2017),7.7,67,414278
5,Thor: Ragnarok,(2017),7.9,74,391485
6,Homem-Aranha: De Volta ao Lar,(2017),7.5,73,361415
7,Corra!,(I) (2017),7.7,84,335393
8,Blade Runner 2049,(2017),8.0,81,334753
9,Em Ritmo de Fuga,(2017),7.6,86,326883


Tudo correu como esperado!

Como observação, se você executar o código de um país onde o inglês não é o idioma principal, é muito provável que você obtenha alguns dos nomes dos filmes traduzidos para o idioma principal desse país.

Provavelmente, isso acontece porque o servidor infere sua localização do seu endereço IP. Mesmo se você estiver em um país onde o inglês é o idioma principal, talvez você ainda receba conteúdo traduzido. Isso pode acontecer se você estiver usando uma VPN enquanto faz as GETsolicitações.

Se você encontrar esse problema, passe os seguintes valores para o headers parâmetro da função get():


# O script para várias páginas
Raspagem de várias páginas é um pouco mais desafiador. Nós vamos construir sobre o nosso script de uma página, fazendo mais três coisas:

Fazendo todos os pedidos que queremos dentro do loop.
Controlando a taxa do loop para evitar bombardear o servidor com solicitações.
Monitorando o loop enquanto ele é executado.
Nós vamos raspar as primeiras 4 páginas de cada ano no intervalo de 2000-2017. 4 páginas para cada um dos 18 anos perfazem um total de 72 páginas. Cada página tem 50 filmes, por isso vamos buscar dados para 3600 filmes no máximo. Mas nem todos os filmes têm um Metascore, então o número será menor do que isso. Mesmo assim, ainda estamos muito propensos a obter dados para mais de 2000 filmes.

# Alterando os parâmetros da URL
Como mostrado anteriormente, as URLs seguem uma certa lógica conforme as páginas da web mudam.

<br>
<img src='https://www.dataquest.io/blog/content/images/url.png'>
<br>

Como estamos fazendo as solicitações, precisaremos apenas variar os valores de apenas dois parâmetros da URL: o release_dateparâmetro e page. Vamos preparar os valores que precisaremos para o próximo ciclo. Na próxima célula de código nós iremos:

- Crie uma lista chamada pages e preencha-a com as sequências correspondentes às primeiras 4 páginas.
- Crie uma lista chamada years_url e preencha-a com as strings correspondentes aos anos 2000-2017.

In [34]:
pages = [str(i) for i in range(1,5)]
pages

['1', '2', '3', '4']

In [35]:
years_url = [str(i) for i in range(2015,2018)]
years_url

['2015', '2016', '2017']

# Controlando a crawl-rate

Controlar a taxa de rastreamento é benéfico para nós e para o site que estamos raspando. Se evitarmos martelar o servidor com dezenas de solicitações por segundo, é muito menos provável que nosso endereço IP seja banido. Também evitamos interromper a atividade do site que criamos, permitindo que o servidor responda também às solicitações de outros usuários.

Controlaremos a taxa do loop usando a sleep() função do time módulo do Python. sleep() fará uma pausa na execução do loop por um período especificado de segundos.

Para imitar o comportamento humano, vamos variar a quantidade de tempo de espera entre as solicitações usando a randint() função do random módulo do Python . randint()aleatoriamente gera inteiros dentro de um intervalo especificado.

<br>
<img src='https://www.dataquest.io/blog/content/images/2017/12/sleep_new.gif'>
<br>

Por enquanto, vamos apenas importar essas duas funções para evitar superlotação na célula de código que contém nosso loop principal.

In [29]:
from time import sleep
from random import randint

# Monitorando o loop como ele ainda está indo
Dado que estamos raspando 72 páginas, seria bom se pudéssemos encontrar uma maneira de monitorar o processo de raspagem como ele ainda está indo. Esse recurso é definitivamente opcional, mas pode ser muito útil no processo de teste e depuração. Além disso, quanto maior o número de páginas, mais útil será o monitoramento. Se você for raspar centenas ou milhares de páginas da web em uma única execução de código, eu diria que esse recurso se torna uma obrigação.

Para nosso script, usaremos esse recurso e monitoraremos os seguintes parâmetros:

- A frequência (velocidade) das solicitações , portanto, garantimos que nosso programa não sobrecarregue o servidor.
- O número de pedidos , para que possamos interromper o loop caso o número de solicitações esperadas seja excedido.
- O código de status de nossas solicitações, portanto, garantimos que o servidor esteja enviando as respostas adequadas.

Para obter um valor de frequência, dividimos o número de solicitações pelo tempo decorrido desde a primeira solicitação. Isso é semelhante ao cálculo da velocidade de um carro - dividimos a distância pelo tempo gasto para cobrir essa distância. Vamos experimentar primeiro essa técnica de monitoramento em pequena escala. Na seguinte célula de código nós iremos:

1. Defina uma hora de início usando a time() função do time módulo e atribua o valor a start_time.
2. Atribua 0 à variável requestsque usaremos para contar o número de solicitações.
3. Inicie um loop e, em seguida, a cada iteração:
4. Simule um pedido.
5. Incrementar o número de solicitações por 1.
6. Pause o loop por um intervalo de tempo entre 8 e 15 segundos.
7. Calcule o tempo decorrido desde a primeira solicitação e atribua o valor a elapsed_time.
8. Imprima o número de pedidos e a frequência.

In [30]:
from time import time

start_time = time()
requests = 0

for _ in range(5):
    # A request would go here
    requests += 1
    sleep(randint(1,3))
    elapsed_time = time() - start_time
    print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))

Request: 1; Frequency: 0.33328382909484083 requests/s
Request: 2; Frequency: 0.3999363619022528 requests/s
Request: 3; Frequency: 0.37496020694682186 requests/s
Request: 4; Frequency: 0.3635581476434034 requests/s
Request: 5; Frequency: 0.3845302569915305 requests/s


Como vamos fazer 72 requests, nosso trabalho parecerá um pouco desordenado à medida que a saída se acumular. Para evitar isso, limparemos a saída após cada iteração e a substituiremos por informações sobre a solicitação mais recente. Para fazer isso, usaremos a clear_output()função do core.display módulo do IPython . Vamos definir o parâmetro wait de clear_output() para True esperar com a substituição da saída atual até que apareça alguma nova saída.

In [31]:
from IPython.core.display import clear_output

start_time = time()
requests = 0

for _ in range(5):
    # A request would go here
    requests += 1
    sleep(randint(1,3))
    current_time = time()
    elapsed_time = current_time - start_time
    print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

Request: 5; Frequency: 0.5540832007782495 requests/s


Para monitorar o código de status, definiremos o programa para nos avisar se algo estiver desligado. Uma solicitação bem-sucedida é indicada por um código de status de 200. Usaremos a warn() função do warnings módulo para lançar um aviso se o código de status não for 200.



In [32]:
from warnings import warn

warn("Warning Simulation")

C:\Users\sn3fru\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Warning Simulation
  This is separate from the ipykernel package so we can avoid doing imports until


Escolhemos um aviso sobre a quebra do loop, porque há uma boa possibilidade de obter dados suficientes, mesmo que algumas das solicitações falhem. Vamos apenas quebrar o loop se o número de solicitações for maior que o esperado.

# Juntando tudo
Agora vamos juntar tudo o que fizemos até agora! Na célula de código a seguir, começamos por:

- Redeclarando as variáveis de listas para que elas se tornem vazias novamente.
- Preparando o monitoramento do loop.

**Então nós vamos:**

    Percorra a years_urllista para variar o release_dateparâmetro do URL.
    Para cada elemento years_url, percorra a pageslista para variar o pageparâmetro da URL.
    Faça os GETpedidos dentro do pagesloop.
    Pause o loop por um intervalo de tempo entre 8 e 15 segundos.
    Monitore cada solicitação como discutido anteriormente.
    Lance um aviso para códigos de status não-200.
    Quebre o loop se o número de solicitações for maior que o esperado.
    Converta o responseconteúdo HTML de um BeautifulSoupobjeto.
    Extraia todos os contêineres de filmes desse BeautifulSoupobjeto.
    Faça um loop por todos esses contêineres.
    Extraia os dados se um contêiner tiver um Metascore.

In [36]:
# Redeclaring the lists to store data in
names = []
years = []
imdb_ratings = []
metascores = []
votes = []

# Preparing the monitoring of the loop
start_time = time()
requests = 0

# For every year in the interval 2015-2017
for year_url in years_url:

    # For every page in the interval 1-4
    for page in pages:

        # Make a get request
        response = get('http://www.imdb.com/search/title?release_date=' \
                       + str(year_url) + 
        '&sort=num_votes,desc&page=' + str(page))

        # Pause the loop
        sleep(randint(1,4))

        # Monitor the requests
        requests += 1
        elapsed_time = time() - start_time
        print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
        clear_output(wait = True)

        # Throw a warning for non-200 status codes
        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests, response.status_code))

        # Break the loop if the number of requests is greater than expected
        if requests > 72:
            warn('Number of requests was greater than expected.')  
            break 

        # Parse the content of the request with BeautifulSoup
        page_html = BeautifulSoup(response.text, 'html.parser')

        # Select all the 50 movie containers from a single page
        mv_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')

        # For every movie of these 50
        for container in mv_containers:
            # If the movie has a Metascore, then:
            if container.find('div', class_ = 'ratings-metascore') is not None:

                # Scrape the name
                name = container.h3.a.text
                names.append(name)

                # Scrape the year 
                year = container.h3.find('span', class_ = 'lister-item-year').text
                years.append(year)

                # Scrape the IMDB rating
                imdb = float(container.strong.text)
                imdb_ratings.append(imdb)

                # Scrape the Metascore
                m_score = container.find('span', class_ = 'metascore').text
                metascores.append(int(m_score))

                # Scrape the number of votes
                vote = container.find('span', attrs = {'name':'nv'})['data-value']
                votes.append(int(vote))

Request:12; Frequency: 0.20855551143699125 requests/s


# Examinando os dados raspados
No próximo bloco de código nós:

Mesclar os dados em um pandas DataFrame.
Imprima algumas informações sobre o recém criado DataFrame.
Mostre as 10 primeiras entradas.

In [37]:
movie_ratings = pd.DataFrame({'movie': names,
                              'year': years,
                              'imdb': imdb_ratings,
                              'metascore': metascores,
                              'votes': votes})

movie_ratings.head(10)

,movie,year,imdb,metascore,votes
0,Star Wars: O Despertar da Força,(2015),8.0,81,755371
1,Mad Max: Estrada da Fúria,(2015),8.1,90,737317
2,Perdido em Marte,(2015),8.0,80,638088
3,Vingadores: Era de Ultron,(2015),7.4,66,607093
4,O Regresso,(2015),8.0,76,578145
5,Jurassic World: O Mundo dos Dinossauros,(2015),7.0,59,515548
6,Divertida Mente,(I) (2015),8.2,94,500701
7,Homem-Formiga,(2015),7.3,64,452080
8,Os Oito Odiados,(2015),7.8,68,405124
9,007 Contra Spectre,(I) (2015),6.8,60,339558


In [38]:
movie_ratings.shape

(454, 5)

# Exemplo 2

Utilizando bs4 para pegar dados do campeonato Argentino.

Nesta Prática Guiada usaremos BeautifulSoup para baixar a informação da primeira divisão de futebol da Argentina da página da ESPN:

    http://www.espn.com.ar/futbol/posiciones/_/liga/arg.

Devemos baixá-la, extraí-la, ordená-la e guardá-la em um csv e/ou um DataFrame para seu uso. 
Primeiro, importamos as bibliotecas necessárias.

In [39]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#Com isto, podemos ver o código HTML diretamente no Notebook
from IPython.display import HTML, display

Para verificar se não há problema scrapeando esta informação, revisamos o arquivo "robots.txt"

In [2]:
robots = 'http://www.espn.com.ar/' + 'robots.txt'
print(requests.get(robots).text)

User-agent: *
Disallow: *print?id
Disallow: /format/
Disallow: /nfl/format/player/design09/dropdown
Disallow: /nba/format/player/design09/dropdown
Disallow: /golf/deportes/format/player/design09/dropdown
Disallow: /extra/mma/deportes/format/player/design09/dropdown
Disallow: /extra/mma/format/player/design09/dropdown
Disallow: /rpm/deportes/format/player/design09/dropdown
Disallow: /golf/format/player/design09/dropdown
Disallow: /mlb/format/player/design09/dropdown
Disallow: /futbol/copamundial2010/
Disallow: /members/v3_1/login
Disallow: /insider/
Disallow: /videohub/video/clip?id
Disallow: /futbolint/noticias/
Disallow: /futbolargentino/columnas/
Disallow: /blogs/null
Disallow: /personalization/v3_1/personalization
Disallow: /futbol/equipo/null
Disallow: /mundial2006/
Disallow: /webslices/




Nossa URL não aparece, então procedemos.

Fazemos um request "GET" ao URL com a tabela e vemos o conteúdo da resposta no Notebook

In [40]:
url = 'http://www.espn.com.ar/futbol/posiciones/_/liga/arg.1'
resp = requests.get(url)

In [41]:
#Vemos o conteúdo renderizado
# display(HTML(resp.text))

In [42]:
soup = BeautifulSoup(resp.text, 'html.parser')

Podemos acessar "tags" diretamente

In [43]:
soup.title

<title>Posiciones de la Superliga Argentina</title>

Para trabalhar com o conteúdo, vamos usar alguns dos seguintes métodos que este objeto tem, como:

- .findAll() / .find_all
- .find()
- .get()
- .get_text()

In [44]:
soup.head.link.get('href')

'http://www.espn.com.ar/futbol/posiciones/_/liga/arg.1'

In [45]:
#Podemos usar find() para encontrar a tabela

raw_table = soup.find('table', {'class':'standings has-team-logos'})

In [47]:
type(raw_table)

bs4.element.Tag

Isto nos retorna um objeto "Tag" definido pela biblioteca BeautifulSoup. Podemos ver seus atributos usando .attrs

In [48]:
raw_table.attrs

{'cellspacing': '0',
 'cellpadding': '0',
 'data-text-contract': 'Contract table',
 'data-text-expand': 'Expand table',
 'data-fix-cols': '1',
 'data-mobile-force-responsive': 'true',
 'data-behavior': 'responsive_table',
 'data-set-cell-heights': 'false',
 'class': ['standings', 'has-team-logos']}

Dentro do atributo "children" encontramos todos os descendentes desse nó. No caso da nossa tabela, os descendentes são tags "tr", que correspondem a cada fila. Existe outro atributo chamado "descendants", que, ao contrário do primeiro, é recursivo. Neste caso, só precisamos dos nós diretamente próximos, então usamos o primeiro método.

Iteramos todas as filas e formamos uma matriz com os dados, para depois carregá-los a um DataFrame.

In [49]:
rows = []
for row in raw_table.children:
    rows.append(row.get_text(separator= ','))

rows

['',
 '',
 '',
 '',
 '2018/19 Superliga,PJ,G,E,P,GF,GC,DIF,PTS',
 '1,Racing Club,RAC,8,6,2,0,15,4,+11,20',
 '2,Atlético Tucumán,CAT,7,4,3,0,13,6,+7,15',
 '3,Unión de Santa Fe,USF,8,4,3,1,10,4,+6,15',
 '4,Aldosivi,ALDO,8,5,0,3,9,6,+3,15',
 '5,Huracán,HUR,7,4,2,1,9,3,+6,14',
 '6,Boca Juniors,CABJ,8,4,2,2,10,7,+3,14',
 '7,River Plate,CARP,7,3,4,0,12,3,+9,13',
 '8,Defensa y Justicia,DYJ,6,3,3,0,7,4,+3,12',
 '9,Banfield,BAN,8,3,3,2,7,7,0,12',
 '10,Gimnasia La Plata,GLP,8,3,2,3,6,6,0,11',
 '11,Vélez Sarsfield,VEL,8,3,2,3,8,10,-2,11',
 '12,Independiente,IND,7,2,4,1,10,7,+3,10',
 '13,Talleres de Córdoba,TDC,8,3,1,4,9,7,+2,10',
 '14,Tigre,TIG,8,2,4,2,9,11,-2,10',
 '15,Godoy Cruz de Mendoza,GCM,8,3,1,4,5,7,-2,10',
 '16,Rosario Central,ROS,7,3,1,3,5,8,-3,10',
 '17,San Martín de San Juan,SMSJ,7,2,2,3,9,12,-3,8',
 '18,San Lorenzo,SL,7,1,4,2,9,11,-2,7',
 '19,Colón de Santa Fe,COL,8,1,4,3,7,11,-4,7',
 '20,Belgrano de Córdoba,BEL,8,1,4,3,3,8,-5,7',
 "21,Newell's Old Boys,NOB,7,1,3,3,6,8,-2,6",
 '22,Ar

In [50]:
#Separamos as colunas e descartamos as filas vazias
table = [row.split(',') for row in rows if len(row) > 1]

In [56]:
pd.DataFrame(table)

,0,1,2,3,4,5,6,7,8,9,10
0,2018/19 Superliga,PJ,G,E,P,GF,GC,DIF,PTS,None,None
1,1,Racing Club,RAC,8,6,2,0,15,4,+11,20
2,2,Atlético Tucumán,CAT,7,4,3,0,13,6,+7,15
3,3,Unión de Santa Fe,USF,8,4,3,1,10,4,+6,15
4,4,Aldosivi,ALDO,8,5,0,3,9,6,+3,15
5,5,Huracán,HUR,7,4,2,1,9,3,+6,14
6,6,Boca Juniors,CABJ,8,4,2,2,10,7,+3,14
7,7,River Plate,CARP,7,3,4,0,12,3,+9,13
8,8,Defensa y Justicia,DYJ,6,3,3,0,7,4,+3,12
9,9,Banfield,BAN,8,3,3,2,7,7,0,12


In [57]:
#Corrigimos a tabela

table[0] = ['Index', 'Name', 'Abbr'] + table[0][1:]
print(*table, sep = '\n')

['Index', 'Name', 'Abbr', 'PJ', 'G', 'E', 'P', 'GF', 'GC', 'DIF', 'PTS']
['1', 'Racing Club', 'RAC', '8', '6', '2', '0', '15', '4', '+11', '20']
['2', 'Atlético Tucumán', 'CAT', '7', '4', '3', '0', '13', '6', '+7', '15']
['3', 'Unión de Santa Fe', 'USF', '8', '4', '3', '1', '10', '4', '+6', '15']
['4', 'Aldosivi', 'ALDO', '8', '5', '0', '3', '9', '6', '+3', '15']
['5', 'Huracán', 'HUR', '7', '4', '2', '1', '9', '3', '+6', '14']
['6', 'Boca Juniors', 'CABJ', '8', '4', '2', '2', '10', '7', '+3', '14']
['7', 'River Plate', 'CARP', '7', '3', '4', '0', '12', '3', '+9', '13']
['8', 'Defensa y Justicia', 'DYJ', '6', '3', '3', '0', '7', '4', '+3', '12']
['9', 'Banfield', 'BAN', '8', '3', '3', '2', '7', '7', '0', '12']
['10', 'Gimnasia La Plata', 'GLP', '8', '3', '2', '3', '6', '6', '0', '11']
['11', 'Vélez Sarsfield', 'VEL', '8', '3', '2', '3', '8', '10', '-2', '11']
['12', 'Independiente', 'IND', '7', '2', '4', '1', '10', '7', '+3', '10']
['13', 'Talleres de Córdoba', 'TDC', '8', '3', '1', '4

Para guardar o arquivo a um csv usando Python, usamos:

In [58]:
df = pd.DataFrame(table[1:], columns= table[0])
df

,Index,Name,Abbr,PJ,G,E,P,GF,GC,DIF,PTS
0,1,Racing Club,RAC,8,6,2,0,15,4,+11,20
1,2,Atlético Tucumán,CAT,7,4,3,0,13,6,+7,15
2,3,Unión de Santa Fe,USF,8,4,3,1,10,4,+6,15
3,4,Aldosivi,ALDO,8,5,0,3,9,6,+3,15
4,5,Huracán,HUR,7,4,2,1,9,3,+6,14
5,6,Boca Juniors,CABJ,8,4,2,2,10,7,+3,14
6,7,River Plate,CARP,7,3,4,0,12,3,+9,13
7,8,Defensa y Justicia,DYJ,6,3,3,0,7,4,+3,12
8,9,Banfield,BAN,8,3,3,2,7,7,0,12
9,10,Gimnasia La Plata,GLP,8,3,2,3,6,6,0,11


In [30]:
filename = 'tabla.csv'
with open(filename, 'w') as out:
    out.write('\n'.join([','.join(row) for row in table]))

In [32]:
df.columns

Index(['Index', 'Name', 'Abbr', 'PJ', 'G', 'E', 'P', 'GF', 'GC', 'DIF', 'PTS'], dtype='object')

In [ ]:
df.set_index('Index', inplace=True)
df.head()

Por último, podemos guardar o arquivo usando pandas com:

In [35]:
filename = 'tabela_ex.xlsx'
df.to_excel(filename)

# Exercicio

Captura uma lista de noticias **e** urls das noticias da pagina principal do site de noticias.

obs: tente não acessar os href para não bloquearem nosso ip ;)

# Desafio

Crie um programa que pegue os preços dos notebooks diariamente no site walmart e avise quando o preço diminuir mais que 10%.